# Setup for TEtranscripts pipeline on google cloud
https://github.com/mhammell-laboratory/TEtranscripts

In [5]:
WRKDIR = '/local/directory/for/files'
DOWNLOADDIR = '{}/pipelinedownloads'.format(WRKDIR)
GCOUTPUT = 'gs://bucketname/output'
USER = 'myuser'
PROJECT_ID = 'projectid'
GCRESOURCES = 'gs://bucketname/TEtranscriptsresources'
COHORT = 'cohort'

# (1) Get gene annotation file
### download the hg38 gene annotation file: http://genome.ucsc.edu/cgi-bin/hgTables 
### enter file name, select gtf output format, and hit get output

### run the following to push to google cloud

In [32]:
print('copy and run locally:')
print('gsutil cp {}/hg38_gene_annotation.gtf {}/'.format(DOWNLOADDIR, GCRESOURCES))

copy and run locally:
gsutil cp /local/path/for/files/pipelinedownloads/hg38_gene_annotation.gtf gs://bucketname/TEtranscriptsresources/


# (2) Use (a) provided transposable element annotation file or (b) generate own from DFAM data or (c) make custom from lifted over hg19 elements

### (a) Download the transposable elements annotation file and push to google cloud (if not generating on your own)

In [33]:
print('copy and run locally:')
print("curl http://labshare.cshl.edu/shares/mhammelllab/www-data/TEtranscripts/TE_GTF/hg38_rmsk_TE.gtf.gz -o {}/hg38_rmsk_TE.gtf.gz".format(DOWNLOADDIR))
print("gunzip {}/hg38_rmsk_TE.gtf.gz".format(DOWNLOADDIR))
print("gsutil cp {}/hg38_rmsk_TE.gtf {}/".format(DOWNLOADDIR, GCRESOURCES))

copy and run locally:
curl http://labshare.cshl.edu/shares/mhammelllab/www-data/TEtranscripts/TE_GTF/hg38_rmsk_TE.gtf.gz -o /local/path/for/files/pipelinedownloads/hg38_rmsk_TE.gtf.gz
gunzip /local/path/for/files/pipelinedownloads/hg38_rmsk_TE.gtf.gz
gsutil cp /local/path/for/files/pipelinedownloads/hg38_rmsk_TE.gtf gs://bucketname/TEtranscriptsresources/


### (b) Build full hg38 transposable element tsv annotation file to pass to PERL script that creates the GTF we want

In [34]:

print("curl https://www.dfam.org/releases/Dfam_3.1/annotations/hg38/hg38_dfam.nrph.hits.gz -o {}/hg38_dfam.nrph.hits.gz".format(DOWNLOADDIR))

print("gunzip {}/hg38_dfam.nrph.hits.gz".format(DOWNLOADDIR))

curl https://www.dfam.org/releases/Dfam_3.1/annotations/hg38/hg38_dfam.nrph.hits.gz -o /local/path/for/files/pipelinedownloads/hg38_dfam.nrph.hits.gz
gunzip /local/path/for/files/pipelinedownloads/hg38_dfam.nrph.hits.gz


read the hits file and check the size

In [3]:
import pandas as pd

hits = pd.read_csv("{}/hg38_dfam.nrph.hits".format(DOWNLOADDIR), sep='\t')

print(hits.shape)
hits.head()

(5657545, 15)


,#seq_name,family_acc,family_name,bits,e-value,bias,hmm-st,hmm-en,strand,ali-st,ali-en,env-st,env-en,sq-len,kimura_div
0,chr1,DF0001137.4,TAR1,355.0,1.100000e-103,98.4,1206,1716,-,10954,10464,10964,10448,248956422,8.32
1,chr1,DF0001137.4,TAR1,558.7,4.100000e-165,71.5,466,1114,-,11463,10826,11482,10810,248956422,6.74
2,chr1,DF0000279.4,L1MC4a_3end,64.0,6.900000e-16,9.4,195,395,-,11676,11502,11696,11480,248956422,17.98
3,chr1,DF0000878.4,MER5B,35.7,1.300000e-05,7.2,1,105,-,11780,11677,11780,11657,248956422,22.20
4,chr1,DF0001253.2,MIR1_Amn,30.9,3.100000e-04,6.8,57,149,-,15353,15265,15376,15248,248956422,18.87


get the unique accession names

In [4]:
import numpy as np
family_acc = hits['family_acc']

print(family_acc.size)
print(family_acc.head())

family_acc = np.unique(family_acc)

print(family_acc.size)
print(family_acc[:10])



5657545
0    DF0001137.4
1    DF0001137.4
2    DF0000279.4
3    DF0000878.4
4    DF0001253.2
Name: family_acc, dtype: object
1304
['DF0000001.4' 'DF0000002.4' 'DF0000003.4' 'DF0000004.4' 'DF0000005.4'
 'DF0000006.4' 'DF0000007.4' 'DF0000008.4' 'DF0000010.4' 'DF0000011.4']


make a data frame to contain the accession names, family ids and class ids

In [5]:
df = pd.DataFrame(data=family_acc)

df.columns = ['DF']
print(df.shape)
print(df.head())

    

(1304, 1)
            DF
0  DF0000001.4
1  DF0000002.4
2  DF0000003.4
3  DF0000004.4
4  DF0000005.4


use the [dfam api](https://www.dfam.org/help/api) to get the family id and class id for each accession name

In [214]:
import requests

url = "https://dfam.org/api/families"

df['family_id'] = [""] * len(df.index)
df['class_id'] = [""] * len(df.index)

print(df.head())
for row in df.itertuples():
    params = {
    "format": "full",
    "name_accession": row.DF.split(".",1)[0]

    }
    
    response = requests.get(url, params=params)
    results = response.json()["results"]
    print(row.Index)
    
    #repeat_subtype_name corresponds to family_id
    df.loc[row.Index,"family_id"] = results[0]["repeat_subtype_name"]

    #repeat_type_name corresponds to class_id
    df.loc[row.Index, "class_id"] = results[0]["repeat_type_name"]
    
    #if either is none then set it to some no_id value
    if(results[0]["repeat_subtype_name"] is None):
        df.loc[row.Index,"family_id"] = "no_family_id"
    
    if(results[0]["repeat_type_name"] is None):
        df.loc[row.Index, "class_id"] = "no_class_id"
    #if(len(results)!=1):
        #break
        
    print(df.loc[[row.Index]])
    
    
print("done")
print(df.shape)

            DF family_id class_id
0  DF0000001.4                   
1  DF0000002.4                   
2  DF0000003.4                   
3  DF0000004.4                   
4  DF0000005.4                   
0
            DF family_id class_id
0  DF0000001.4       MIR     SINE
1
            DF family_id class_id
1  DF0000002.4       Alu     SINE
2
            DF family_id class_id
2  DF0000003.4       Alu     SINE
3
            DF     family_id class_id
3  DF0000004.4  TcMar-Tigger      DNA
4
            DF  family_id class_id
4  DF0000005.4  ERVL-MaLR      LTR
5
            DF family_id class_id
5  DF0000006.4  Helitron       RC
6
            DF family_id class_id
6  DF0000007.4       Alu     SINE
7
            DF family_id class_id
7  DF0000008.4        L1     LINE
8
            DF family_id class_id
8  DF0000010.4        L1     LINE
9
            DF                                      family_id class_id
9  DF0000011.4  root;Interspersed_Repeat;Pseudogene;RNA;snRNA    snRNA
10
         

101
              DF family_id class_id
101  DF0000110.4       CR1     LINE
102
              DF                     family_id   class_id
102  DF0000111.4  root;Tandem_Repeat;Satellite  Satellite
103
              DF  family_id class_id
103  DF0000113.4  TcMar-Tc1      DNA
104
            DF family_id class_id
104  DF0000114      ERV1      LTR
105
            DF family_id class_id
105  DF0000115      ERV1      LTR
106
              DF family_id class_id
106  DF0000116.4      ERVL      LTR
107
              DF family_id class_id
107  DF0000117.4      ERVL      LTR
108
              DF family_id class_id
108  DF0000118.4      ERVL      LTR
109
              DF family_id class_id
109  DF0000119.4      ERVL      LTR
110
              DF family_id class_id
110  DF0000120.4      ERVL      LTR
111
              DF                                          family_id class_id
111  DF0000121.4  root;Interspersed_Repeat;Transposable_Element;...      DNA
112
              DF                        

195
              DF                     family_id   class_id
195  DF0000209.4  root;Tandem_Repeat;Satellite  Satellite
196
              DF                     family_id   class_id
196  DF0000210.4  root;Tandem_Repeat;Satellite  Satellite
197
              DF                     family_id   class_id
197  DF0000211.4  root;Tandem_Repeat;Satellite  Satellite
198
              DF      family_id class_id
198  DF0000212.4  TcMar-Mariner      DNA
199
              DF      family_id class_id
199  DF0000213.4  TcMar-Mariner      DNA
200
            DF family_id class_id
200  DF0000214      ERV1      LTR
201
              DF family_id class_id
201  DF0000215.4      ERV1      LTR
202
            DF family_id class_id
202  DF0000216      ERV1      LTR
203
            DF family_id class_id
203  DF0000217      ERV1      LTR
204
              DF  family_id class_id
204  DF0000218.4  TcMar-Tc2      DNA
205
              DF  family_id class_id
205  DF0000219.4  TcMar-Tc2      DNA
206
              DF

301
              DF family_id class_id
301  DF0000315.4        L1     LINE
302
              DF family_id class_id
302  DF0000316.4        L1     LINE
303
              DF family_id class_id
303  DF0000317.4        L1     LINE
304
              DF family_id class_id
304  DF0000318.4        L1     LINE
305
              DF family_id class_id
305  DF0000319.4        L1     LINE
306
              DF family_id class_id
306  DF0000320.4        L1     LINE
307
              DF family_id class_id
307  DF0000321.4        L1     LINE
308
              DF family_id class_id
308  DF0000322.4        L1     LINE
309
              DF family_id class_id
309  DF0000323.4        L1     LINE
310
              DF family_id class_id
310  DF0000324.4        L1     LINE
311
              DF family_id class_id
311  DF0000325.4        L1     LINE
312
              DF family_id class_id
312  DF0000326.4        L1     LINE
313
              DF family_id class_id
313  DF0000327.4        L1     LINE
314
        

407
              DF family_id class_id
407  DF0000421.4      ERVL      LTR
408
              DF family_id class_id
408  DF0000422.4      ERVL      LTR
409
              DF family_id class_id
409  DF0000423.4      ERVL      LTR
410
              DF family_id class_id
410  DF0000424.4      ERVL      LTR
411
              DF family_id class_id
411  DF0000425.4      ERVL      LTR
412
              DF family_id class_id
412  DF0000426.4      ERVL      LTR
413
              DF family_id class_id
413  DF0000427.4      ERV1      LTR
414
              DF family_id class_id
414  DF0000428.4      ERVL      LTR
415
              DF family_id class_id
415  DF0000429.4      ERVL      LTR
416
              DF family_id class_id
416  DF0000430.4      ERVL      LTR
417
              DF family_id class_id
417  DF0000431.4      ERV1      LTR
418
              DF family_id class_id
418  DF0000432.4      ERV1      LTR
419
              DF family_id class_id
419  DF0000433.4      ERV1      LTR
420
        

...

877
              DF family_id class_id
877  DF0000897.4      ERV1      LTR
878
              DF family_id class_id
878  DF0000898.4      ERV1      LTR
879
              DF family_id class_id
879  DF0000899.4      ERV1      LTR
880
              DF family_id class_id
880  DF0000900.4      ERV1      LTR
881
              DF family_id class_id
881  DF0000901.4      ERV1      LTR
882
              DF family_id class_id
882  DF0000902.4      ERV1      LTR
883
              DF family_id class_id
883  DF0000903.4      ERV1      LTR
884
              DF family_id class_id
884  DF0000904.4      ERV1      LTR
885
              DF family_id class_id
885  DF0000905.4      ERV1      LTR
886
              DF family_id class_id
886  DF0000906.4      ERV1      LTR
887
              DF family_id class_id
887  DF0000907.4      ERV1      LTR
888
              DF family_id class_id
888  DF0000908.4      ERV1      LTR
889
              DF family_id class_id
889  DF0000909.4      ERV1      LTR
890
        

979
              DF   family_id class_id
979  DF0000999.4  hAT-Tip100      DNA
980
              DF family_id class_id
980  DF0001000.4    Merlin      DNA
981
              DF  family_id class_id
981  DF0001001.4  ERVL-MaLR      LTR
982
              DF  family_id class_id
982  DF0001002.4  ERVL-MaLR      LTR
983
              DF  family_id class_id
983  DF0001003.4  ERVL-MaLR      LTR
984
              DF  family_id class_id
984  DF0001004.4  ERVL-MaLR      LTR
985
              DF  family_id class_id
985  DF0001005.4  ERVL-MaLR      LTR
986
              DF  family_id class_id
986  DF0001006.4  ERVL-MaLR      LTR
987
              DF  family_id class_id
987  DF0001007.4  ERVL-MaLR      LTR
988
              DF  family_id class_id
988  DF0001008.4  ERVL-MaLR      LTR
989
              DF  family_id class_id
989  DF0001009.4  ERVL-MaLR      LTR
990
              DF  family_id class_id
990  DF0001010.4  ERVL-MaLR      LTR
991
              DF  family_id class_id
991  DF0001011.4  ERVL-

1067
               DF                                          family_id class_id
1067  DF0001089.4  root;Interspersed_Repeat;Transposable_Element;...      DNA
1068
               DF family_id class_id
1068  DF0001090.4    hAT-Ac      DNA
1069
               DF                         family_id class_id
1069  DF0001091.4  root;Interspersed_Repeat;Unknown  Unknown
1070
               DF                         family_id class_id
1070  DF0001092.4  root;Interspersed_Repeat;Unknown  Unknown
1071
               DF                         family_id class_id
1071  DF0001093.4  root;Interspersed_Repeat;Unknown  Unknown
1072
               DF                         family_id class_id
1072  DF0001094.4  root;Interspersed_Repeat;Unknown  Unknown
1073
               DF                                          family_id class_id
1073  DF0001095.4  root;Interspersed_Repeat;Transposable_Element;...      DNA
1074
               DF                         family_id class_id
1074  DF0001096.4  root;I

1146
               DF family_id class_id
1146  DF0001171.2     Gypsy      LTR
1147
               DF family_id class_id
1147  DF0001172.2    hAT-Ac      DNA
1148
               DF                         family_id class_id
1148  DF0001173.2  root;Interspersed_Repeat;Unknown  Unknown
1149
               DF family_id class_id
1149  DF0001174.2       Alu     SINE
1150
               DF family_id class_id
1150  DF0001175.2       hAT      DNA
1151
               DF    family_id class_id
1151  DF0001176.2  hAT-Charlie      DNA
1152
               DF family_id class_id
1152  DF0001177.2  RTE-BovB     LINE
1153
               DF  family_id class_id
1153  DF0001178.2  hAT-hAT19      DNA
1154
               DF    family_id class_id
1154  DF0001179.2  hAT-Charlie      DNA
1155
               DF                                          family_id class_id
1155  DF0001180.2  root;Interspersed_Repeat;Transposable_Element;...      DNA
1156
               DF                         family_id class_id


1230
               DF     family_id class_id
1230  DF0001259.2  TcMar-Tigger      DNA
1231
               DF                         family_id class_id
1231  DF0001260.2  root;Interspersed_Repeat;Unknown  Unknown
1232
               DF    family_id class_id
1232  DF0001261.2  hAT-Charlie      DNA
1233
               DF   family_id class_id
1233  DF0001262.2  hAT-Tip100      DNA
1234
               DF                         family_id class_id
1234  DF0001263.2  root;Interspersed_Repeat;Unknown  Unknown
1235
               DF  family_id class_id
1235  DF0001264.2  MULE-MuDR      DNA
1236
               DF family_id class_id
1236  DF0001265.2       CR1     LINE
1237
               DF                                          family_id class_id
1237  DF0001266.2  root;Interspersed_Repeat;Transposable_Element;...      LTR
1238
               DF                         family_id class_id
1238  DF0001267.2  root;Interspersed_Repeat;Unknown  Unknown
1239
               DF family_id class_id
1

In [215]:
print(df.shape)
print(df.head())

(1304, 3)
            DF     family_id class_id
0  DF0000001.4           MIR     SINE
1  DF0000002.4           Alu     SINE
2  DF0000003.4           Alu     SINE
3  DF0000004.4  TcMar-Tigger      DNA
4  DF0000005.4     ERVL-MaLR      LTR


merge the hits data with the family and class ids

In [217]:

full_hits = pd.merge(hits, df, left_on = "family_acc", right_on = "DF", how = "left")

print("original hits df size:")
print(hits.shape)
print("new hits df size:")
print(full_hits.shape)
print(full_hits.head())


original hits df size:
(5657545, 15)
new hits df size:
(5657545, 18)
  #seq_name   family_acc  family_name   bits        e-value  bias  hmm-st  \
0      chr1  DF0001137.4         TAR1  355.0  1.100000e-103  98.4    1206   
1      chr1  DF0001137.4         TAR1  558.7  4.100000e-165  71.5     466   
2      chr1  DF0000279.4  L1MC4a_3end   64.0   6.900000e-16   9.4     195   
3      chr1  DF0000878.4        MER5B   35.7   1.300000e-05   7.2       1   
4      chr1  DF0001253.2     MIR1_Amn   30.9   3.100000e-04   6.8      57   

   hmm-en strand  ali-st  ali-en  env-st  env-en     sq-len  kimura_div  \
0    1716      -   10954   10464   10964   10448  248956422        8.32   
1    1114      -   11463   10826   11482   10810  248956422        6.74   
2     395      -   11676   11502   11696   11480  248956422       17.98   
3     105      -   11780   11677   11780   11657  248956422       22.20   
4     149      -   15353   15265   15376   15248  248956422       18.87   

            DF   

write to a file

In [218]:
full_hits.to_csv("{}/hg38_dfam_annotation.tsv".format(DOWNLOADDIR), index=False, sep='\t')



run the perl script with the new file

In [ ]:
%%bash -s "$DOWNLOADDIR" "$WRKDIR"
DOWNLOADDIR=${1}
WRKDIR=${2}

perl ${WRKDIR}/makeTEgtf.PERL -c 1 -s 10 -e 11 -o 9 -t 3 -f 17 -C 18 -S 4 ${DOWNLOADDIR}/hg38_dfam_annotation.tsv > ${DOWNLOADDIR}/hg38_dfam_annotation.gtf



### May need to swap start and end positions for lines with "-" strand for STAR alignment to work
run this to switch the start and end position for rows with "-" for strand

In [81]:
#awk 'BEGIN {OFS=FS="\t"} $7=="-" {t = $4; $4 = $5; $5 = t; print; next};{print $0} ' hg38_dfam_annotation.gtf > hg38_dfam_annotation_swap.gtf


push the new gtf file to google cloud

In [35]:
print('run locally:')
print('gsutil cp {}/hg38_dfam_annotation.gtf {}/'.format(DOWNLOADDIR, GCRESOURCES))

run locally:
gsutil cp /local/path/for/files/pipelinedownloads/hg38_dfam_annotation.gtf gs://bucketname/TEtranscriptsresources/


### (c) Create gtf with hg19 HSATII converted to hg38 

1) download hg19 Repeatmasker from [UCSC Table Browser](http://genome.ucsc.edu/cgi-bin/hgTables?hgsid=779592275_N3CYal7CTJjbHIyobHuJvr25ifGf&clade=mammal&org=&db=hg19&hgta_group=rep&hgta_track=rmsk&hgta_table=rmsk&hgta_regionType=genome&position=&hgta_outputType=bed&hgta_outFileName=) with repeats group, repeatmasker track and BED format

In [6]:
print('curl "http://genome.ucsc.edu/cgi-bin/hgTables?hgsid=779592275_N3CYal7CTJjbHIyobHuJvr25ifGf&boolshad.hgta_printCustomTrackHeaders=0&hgta_ctName=tb_rmsk&hgta_ctDesc=table+browser+query+on+rmsk&hgta_ctVis=pack&hgta_ctUrl=&fbQual=whole&fbUpBases=200&fbDownBases=200&hgta_doGetBed=get+BED" -o {}/hg19_rmsk.bed'.format(DOWNLOADDIR))

curl "http://genome.ucsc.edu/cgi-bin/hgTables?hgsid=779592275_N3CYal7CTJjbHIyobHuJvr25ifGf&boolshad.hgta_printCustomTrackHeaders=0&hgta_ctName=tb_rmsk&hgta_ctDesc=table+browser+query+on+rmsk&hgta_ctVis=pack&hgta_ctUrl=&fbQual=whole&fbUpBases=200&fbDownBases=200&hgta_doGetBed=get+BED" -o /local/directory/for/files/pipelinedownloads/hg19_rmsk.bed


2) extract HSATII elements from BED file

In [7]:
print("grep HSATII {}/hg19_rmsk.bed > {}/HSATII_hg19_rmsk.bed".format(DOWNLOADDIR,DOWNLOADDIR))

grep HSATII /local/directory/for/files/pipelinedownloads/hg19_rmsk.bed > /local/directory/for/files/pipelinedownloads/HSATII_hg19_rmsk.bed


3) convert the HSATII hg19 elements to hg38 using the liftover tool: https://genome.ucsc.edu/cgi-bin/hgLiftOver

4) convert the lifted over HSATII file to a gtf file

```awk '{print $1"\t""hg19_rmsk""\t""exon""\t"($2+1)"\t"$3"\t"$5"\t"$6"\t"".""\t""gene_id \""$4"\"; transcript_id \""$4"\";"}' HSATII_liftover.bed > HSATII_liftover.gtf```

5) download hg38 Repeatmasker from [UCSC Table Browser](http://genome.ucsc.edu/cgi-bin/hgTables?hgsid=779592275_N3CYal7CTJjbHIyobHuJvr25ifGf&clade=mammal&org=&db=hg38&hgta_group=rep&hgta_track=rmsk&hgta_table=rmsk&hgta_regionType=genome&position=&hgta_outputType=gtf&hgta_outFileName=) with repeats group, repeatmasker track and GTF format

6) get the elements that do not overlap with HSATII

In [2]:
print("module load bedtools")
print("bedtools intersect -a hg38_rmsk.gtf -b HSATII_liftover.gtf -v > no_overlap.gtf")


module load bedtools
bedtools intersect -a hg38_rmsk.gtf -b HSATII_liftover.gtf -v > no_overlap.gtf


7) merge the no overlap file with the HSATII liftover elements

In [3]:
print("cat no_overlap.gtf HSATII_liftover.gtf > custom_HSATII_annotation.gtf")

cat no_overlap.gtf HSATII_liftover.gtf > custom_HSATII_annotation.gtf


8) push new file to google cloud

In [8]:
print('run locally:')
print('gsutil cp {}/custom_HSATII_annotation.gtf {}/'.format(DOWNLOADDIR, GCRESOURCES))

run locally:
gsutil cp /local/directory/for/files/pipelinedownloads/custom_HSATII_annotation.gtf gs://bucketname/TEtranscriptsresources/


# (3) Setup STAR
(Spliced Transcripts Alignment to a Reference)

indexed reference genome files are on biowulf here: ```/fdb/STAR_current```

run these commands to copy (some of the files are soft linked so pushing the folder to gc may not work)

In [125]:
print("navigate here on biowulf:")
print("cd /fdb/STAR_current/UCSC/hg38")

navigate here on biowulf:
cd /fdb/STAR_current/UCSC/hg38


In [36]:
BIOWULFTEMPDIR = "/biowulf/temporary/directory/for/ref/genome/files"

print("mkdir {}/ref".format(BIOWULFTEMPDIR))
print("cp /fdb/STAR_current/UCSC/hg38/ref/chrLength.txt {}/ref/".format(BIOWULFTEMPDIR))
print("cp /fdb/STAR_current/UCSC/hg38/ref/chrNameLength.txt {}/ref/".format(BIOWULFTEMPDIR))
print("cp /fdb/STAR_current/UCSC/hg38/ref/chrName.txt {}/ref/".format(BIOWULFTEMPDIR))
print("cp /fdb/STAR_current/UCSC/hg38/ref/chrStart.txt {}/ref/".format(BIOWULFTEMPDIR))
print("cp /fdb/STAR_indices/repo/Genome/c33f343609537f34364ba535438dc9237064327b97fcdc76636f9d1c4d08f739 {}/ref/Genome".format(BIOWULFTEMPDIR))
print("cp /fdb/STAR_current/UCSC/hg38/ref/genomeParameters.txt {}/ref/".format(BIOWULFTEMPDIR))
print("cp /fdb/STAR_indices/repo/SA/cdf66cb5d1a4f1346c066c21568c1a3b48ede6b8e89c1afcd06249247a673080 {}/ref/SA".format(BIOWULFTEMPDIR))
print("cp /fdb/STAR_indices/repo/SAindex/552dd6491165ccfb1f9b4236f1664720f47e9adf395a451e657b768b18cbc1a5 {}/ref/SAindex".format(BIOWULFTEMPDIR))
print("compress the folder:")
print("cd {}".format(BIOWULFTEMPDIR))
print("tar czf {}/ref.tar.gz ref".format(BIOWULFTEMPDIR))

mkdir /biowulf/temporary/directory/for/ref/genome/files/ref
cp /fdb/STAR_current/UCSC/hg38/ref/chrLength.txt /biowulf/temporary/directory/for/ref/genome/files/ref/
cp /fdb/STAR_current/UCSC/hg38/ref/chrNameLength.txt /biowulf/temporary/directory/for/ref/genome/files/ref/
cp /fdb/STAR_current/UCSC/hg38/ref/chrName.txt /biowulf/temporary/directory/for/ref/genome/files/ref/
cp /fdb/STAR_current/UCSC/hg38/ref/chrStart.txt /biowulf/temporary/directory/for/ref/genome/files/ref/
cp /fdb/STAR_indices/repo/Genome/c33f343609537f34364ba535438dc9237064327b97fcdc76636f9d1c4d08f739 /biowulf/temporary/directory/for/ref/genome/files/ref/Genome
cp /fdb/STAR_current/UCSC/hg38/ref/genomeParameters.txt /biowulf/temporary/directory/for/ref/genome/files/ref/
cp /fdb/STAR_indices/repo/SA/cdf66cb5d1a4f1346c066c21568c1a3b48ede6b8e89c1afcd06249247a673080 /biowulf/temporary/directory/for/ref/genome/files/ref/SA
cp /fdb/STAR_indices/repo/SAindex/552dd6491165ccfb1f9b4236f1664720f47e9adf395a451e657b768b18cbc1a5 /bi

In [37]:

print("log into google cloud via biowulf:")
print("module load google-cloud-sdk")
print("gcloud auth login")

print("")

print("follow login instructions and then run command to push archived file to google cloud:")

print("gsutil cp \
ref.tar.gz \
{}/".format(GCRESOURCES))

log into google cloud via biowulf:
module load google-cloud-sdk
gcloud auth login

follow login instructions and then run command to push archived file to google cloud:
gsutil cp ref.tar.gz gs://bucketname/TEtranscriptsresources/


# (4) Build and push docker image
docker must be installed locally (https://docs.docker.com/docker-for-mac/install/)

In [3]:
print("run this locally at terminal in folder of dockerfile:")
print("docker build -t tetranscripts-image .")

print("\ntag the image to point it to the google cloud project:")
print("docker tag tetranscripts-image:latest us.gcr.io/{}/tetranscripts-image".format(PROJECT_ID))

print("\npush the image to google cloud:")
print("docker push us.gcr.io/{}/tetranscripts-image".format(PROJECT_ID))

print("\ncheck 'Container Registry' in gcc to find the new image")

run this locally at terminal in folder of dockerfile:
docker build -t tetranscripts-image .

tag the image to point it to the google cloud project:
docker tag tetranscripts-image:latest us.gcr.io/projectid/tetranscripts-image

push the image to google cloud:
docker push us.gcr.io/projectid/tetranscripts-image

check 'Container Registry' in gcc to find the new image


#### once pushed find the container on google cloud and copy its full path
something like:

```us.gcr.io/projectid/test-image@sha256:04bc2af3cccd8618e6eafadc7d46e7fb24b2dc89e0e62ea0bdb26865d081f632```

paste the full path name into the input.json file wherever a docker variable is set

like

```"TEtranscriptsWorkflow.dockerimg": "us.gcr.io/projectid/test-image@sha256:04bc2af3cccd8618e6eafadc7d46e7fb24b2dc89e0e62ea0bdb26865d081f632"```

# (5) Setup Cromwell server for running in parallel

pull down cromwell server setup scripts from gcp

In [39]:
CROMWELL_SCRIPT_BUCKET="gs://somebucketname/tools/verily-amp-pd-source"

In [40]:
print("gsutil -mq cp -r {} {}".format(CROMWELL_SCRIPT_BUCKET,DOWNLOADDIR))


gsutil -mq cp -r gs://somebucketname/tools/verily-amp-pd-source /local/path/for/files/pipelinedownloads


In [41]:
print('\n#fire up the cromwell instance')
print('chmod +x {}/verily-amp-pd-source/setup_cromwell_vm/*.sh'.format(DOWNLOADDIR))
print('cd {}/verily-amp-pd-source/setup_cromwell_vm/'.format(DOWNLOADDIR))
print('./create_cromwell_server.sh {}-cromwell {} n1-highmem-8'\
      .format(COHORT,PROJECT_ID))

print('./configure.sh {}-cromwell {} nihnialngcbg-testing/{}'\
      .format(COHORT,PROJECT_ID,COHORT.replace('_','-')))


#fire up the cromwell instance
chmod +x /local/path/for/files/pipelinedownloads/verily-amp-pd-source/setup_cromwell_vm/*.sh
cd /local/path/for/files/pipelinedownloads/verily-amp-pd-source/setup_cromwell_vm/
./create_cromwell_server.sh cohort-cromwell projectid n1-highmem-8
./configure.sh cohort-cromwell projectid nihnialngcbg-testing/cohort


In [42]:
print('\n#When that is up, ssh to the instance:')
print('gcloud --project {} compute ssh {}-cromwell'.format(PROJECT_ID,COHORT.replace('_','-')))


#When that is up, ssh to the instance:
gcloud --project projectid compute ssh cohort-cromwell


In [43]:
print('\n#And in that SSH session, run:')
print('cd /install')
print('docker-compose -f /install/workspace/config/docker-compose.yml up')


#And in that SSH session, run:
cd /install
docker-compose -f /install/workspace/config/docker-compose.yml up


#### create the ssh tunnel to the cromwell server so you can submit jobs

In [44]:
print('#When cromwell is up, create an SSH tunnel from your workstation:')
print('#run these commands at terminal:\n')
print('gcloud --project {} compute ssh {}-cromwell -- -L 8000:localhost:8000'\
      .format(PROJECT_ID,COHORT.replace('_','-')))

print('\n#after this runs you will actually be logged into the cromwell server, ' \
      'so you will need to open another terminal session on your machine to submit your jobs\n')

#When cromwell is up, create an SSH tunnel from your workstation:
#run these commands at terminal:

gcloud --project projectid compute ssh cohort-cromwell -- -L 8000:localhost:8000

#after this runs you will actually be logged into the cromwell server, so you will need to open another terminal session on your machine to submit your jobs



#  (6) Setup input files

#### get list of sample ids

In [58]:
SAMPLE_BUCKET='gs://some/bucket/with/fastqs'


#### see what fastqs are in the bucket

In [52]:
%%bash -s "$SAMPLE_BUCKET"
SAMPLE_BUCKET=${1}

gsutil ls ${SAMPLE_BUCKET} | wc -l

     622


#### get fastqs listing

In [57]:
%%bash -s "$DOWNLOADDIR" "$SAMPLE_BUCKET" "$COHORT"
DOWNLOADDIR=${1}
SAMPLE_BUCKET=${2}

FASTQ_LISTING=${DOWNLOADDIR}/fastqs.fastq.listing.txt

gsutil ls ${SAMPLE_BUCKET}/*.fastq.gz > ${FASTQ_LISTING}

sed -i -e s"/gs:\/\/some\/bucket\/with\/fastqs\///"g ${FASTQ_LISTING}
sed -i -e s"/\.fastq\.gz//"g ${FASTQ_LISTING}
sed -i -e s'/_R/\,R/'g ${FASTQ_LISTING}

less ${FASTQ_LISTING} | wc -l
head ${FASTQ_LISTING}


     622
KEN1003fctx,R1
KEN1003fctx,R2
KEN1066fctx,R1
KEN1066fctx,R2
KEN1069fctx,R1
KEN1069fctx,R2
KEN1070fctx,R1
KEN1070fctx,R2
KEN1092fctx,R1
KEN1092fctx,R2


In [8]:
import pandas as pd
fastqdf=pd.read_csv("{}/fastqs.fastq.listing.txt".format(DOWNLOADDIR),header=None)
fastqdf.columns = ['ID','READ']
print(fastqdf.shape)
fastqdf.head()


(622, 2)


,ID,READ
0,KEN1003fctx,R1
1,KEN1003fctx,R2
2,KEN1066fctx,R1
3,KEN1066fctx,R2
4,KEN1069fctx,R1


In [9]:
sample_ids = fastqdf['ID'].unique()
print(sample_ids.shape)


(311,)


#### (optional) subset for testing

In [10]:
sample_ids = sample_ids[:3]
print(sample_ids.shape)
print(sample_ids)

(3,)
['KEN1003fctx' 'KEN1066fctx' 'KEN1069fctx']


#### create the per sample json files
### SPECIFY WHICH ANNOTATION TO USE HERE

In [11]:
import json
json_label_template = '{}/templates/blank.tetranscripts.label.json'.format(WRKDIR)
json_input_template = '{}/templates/blank.tetranscripts.input.json'.format(WRKDIR)
json_options_template = '{}/templates/blank.tetranscripts.options.json'.format(WRKDIR)

for sample_id in sample_ids:
    json_label_outfile_name = '{}/jsons/{}.label.json'.format(WRKDIR,sample_id)
    json_input_outfile_name = '{}/jsons/{}.input.json'.format(WRKDIR,sample_id)
    json_options_outfile_name = '{}/jsons/{}.options.json'.format(WRKDIR,sample_id)
    
    with open(json_label_template) as json_file:
        label_data = json.load(json_file)
        
        label_data['cohort'] = COHORT.lower()
        label_data['sample'] = sample_id.lower()
        label_data['user'] = USER.lower()
        
        with open(json_label_outfile_name, 'w') as json_outfile:
            json.dump(label_data,json_outfile,sort_keys=True,indent=4)
            
    with open(json_options_template) as json_file:
        options_data = json.load(json_file)
        
        options_data['final_call_logs_dir'] = "{}/logs/{}".format(GCOUTPUT,sample_id)
        options_data['final_workflow_log_dir'] = "{}/logs/{}".format(GCOUTPUT,sample_id)
        options_data['final_workflow_outputs_dir'] = "{}/output/{}".format(GCOUTPUT,sample_id)
        
        with open(json_options_outfile_name, 'w') as json_outfile:
            json.dump(options_data,json_outfile,sort_keys=True,indent=4)
    with open(json_input_template) as json_file:
        input_data = json.load(json_file)
        
        input_data['TEtranscriptsWorkflow.sample_name'] = sample_id.lower()
        input_data['TEtranscriptsWorkflow.firstPairedEnd'] = "{}/{}_R1.fastq.gz".format(SAMPLE_BUCKET, sample_id)
        input_data['TEtranscriptsWorkflow.secondPairedEnd'] = "{}/{}_R2.fastq.gz".format(SAMPLE_BUCKET, sample_id)
        
        input_data['TEtranscriptsWorkflow.gene_annotation_gtf'] = "{}/hg38_gene_annotation.gtf".format(GCRESOURCES)
        input_data['TEtranscriptsWorkflow.refgenomeFile'] = "{}/ref.tar.gz".format(GCRESOURCES)
        #ANNOTATION TO USE:
        input_data['TEtranscriptsWorkflow.TE_annotation_gtf'] = "{}/custom_HSATII_annotation.gtf".format(GCRESOURCES)
        
        with open(json_input_outfile_name, 'w') as json_outfile:
            json.dump(input_data,json_outfile,sort_keys=True,indent=4)

#### function to format the command

In [12]:

def formatgcpcmd(this_sample):
    this_cmd = 'echo -n {SAMPLE} \n\
python {DOWNLOADDIR}/verily-amp-pd-source/wdl_workflow_runner/cromwell_client.py \
--wdl {WRKDIR}/TEtranscript_count_pipeline.wdl \
--workflow-inputs {WRKDIR}/jsons/{SAMPLE}.input.json \
--workflow-options {WRKDIR}/jsons/{SAMPLE}.options.json \
--workflow-labels {WRKDIR}/jsons/{SAMPLE}.label.json'
    return(this_cmd.format(DOWNLOADDIR=DOWNLOADDIR,WRKDIR=WRKDIR,SAMPLE=this_sample))



#### make a command for each sample and write it to a file

In [64]:
cmds = [formatgcpcmd(sample_id) for sample_id in sample_ids]

temp_script_file = '{}/run_TEtranscripts.sh'.format(WRKDIR)

with open(temp_script_file, 'w') as file_handler:
    for this_cmd in cmds:
        file_handler.write("{}\n".format(this_cmd))
        


# (7) Run TEtranscripts

In [63]:
print('#run these commands at terminal:\n')
print('chmod +x ' + temp_script_file)
print('nohup ' + temp_script_file + ' > {}/run_TEtranscripts.log &'.format(WRKDIR))

#run these commands at terminal:

chmod +x /local/directory/for/files/run_TEtranscripts.sh
nohup /local/directory/for/files/run_TEtranscripts.sh > /local/directory/for/files/run_TEtranscripts.log &


# (8) Monitor the jobs

In [18]:
#see how many GCE (Google Compute Engine) instances are running your jobs


print('#full job worker node count')
!gcloud compute instances list --project {PROJECT_ID} \
--filter "labels:({COHORT} {USER})" | grep RUNNING | wc -l

print('#number of all running instances in project')
!gcloud compute instances list --project {PROJECT_ID} | grep RUNNING | wc -l

#full job worker node count
Listed 0 items.
       0
#number of all running instances in project
       1


In [65]:
print('#run these commands at terminal:\n')

print('#When cromwell is up, create an SSH tunnel from your workstation, if not already connected:\n')
print('gcloud --project {} compute ssh {}-cromwell -- -L 8000:localhost:8000'.\
      format(PROJECT_ID,COHORT.replace('_','-')))

print('\n#if tunnel established can check cromwell status\n')
print('curl -X GET "http://localhost:8000/api/workflows/v1/query?status=Running"')
print('curl -X GET "http://localhost:8000/api/workflows/v1/query?status=Submitted"')
print('curl -X GET "http://localhost:8000/api/workflows/v1/query?status=Failed"')
print('curl -X GET "http://localhost:8000/api/workflows/v1/query?status=Succeeded"')

#run these commands at terminal:

#When cromwell is up, create an SSH tunnel from your workstation, if not already connected:

gcloud --project projectid compute ssh cohort-cromwell -- -L 8000:localhost:8000

#if tunnel established can check cromwell status

curl -X GET "http://localhost:8000/api/workflows/v1/query?status=Running"
curl -X GET "http://localhost:8000/api/workflows/v1/query?status=Submitted"
curl -X GET "http://localhost:8000/api/workflows/v1/query?status=Failed"
curl -X GET "http://localhost:8000/api/workflows/v1/query?status=Succeeded"


# (9) Gather output


In [2]:
LOCAL_OUTPUT="/local/directory/to/store/output"

In [66]:
print("run locally:")
print("gsutil cp -r {}/output/ {}".format(GCOUTPUT, LOCAL_OUTPUT))
print("cd {}".format(LOCAL_OUTPUT))
print("find . -iname '*.cntTable' -exec cp {} . \;")

print("rm -r {}/output/".format(LOCAL_OUTPUT))

run locally:
gsutil cp -r gs://bucketname/output/output/ /Users/grennfp/TEtranscript_project/test_output
cd /Users/grennfp/TEtranscript_project/test_output
find . -iname '*.cntTable' -exec cp {} . \;
rm -r /Users/grennfp/TEtranscript_project/test_output/output/
